In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import numpy

In [2]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=50000, n_features=8, random_state=0)

In [3]:
# Function to create model, required for KerasClassifier
#def create_model(optimizer='adam'):
#    # create model
#    model = Sequential()
#    model.add(Dense(12, input_dim=8, activation='relu'))
#    model.add(Dense(8, activation='relu'))
#    model.add(Dense(1, activation='sigmoid'))
#    # Compile model
#    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
#    return model

In [4]:
# fix random seed for reproducibility
#seed = 7

In [5]:
# load pima indians dataset
#dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")

In [6]:
# split into input (X) and output (Y) variables
#X = dataset[:,0:8]
#y = dataset[:,8]

In [7]:
# create model
#model = create_model()#KerasClassifier(build_fn=create_model, verbose=0)
from sklearn.svm import SVC

estimator = SVC(random_state=0)

In [8]:
# grid search epochs, batch size and optimizer
#optimizers = ['rmsprop', 'adam']
#init = ['glorot_uniform', 'normal', 'uniform']
#epochs = [150]
#batches = [5, 10, 20]
#param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)
#grid = GridSearchCV(estimator=model, param_grid=param_grid)

In [9]:
from sklearn.model_selection import GridSearchCV

In [10]:
%%time
estimator = SVC(gamma='auto', random_state=0, probability=True)
param_grid = {
    'C': [0.001, 10.0],
    'kernel': ['rbf', 'poly'],
}

grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=2, n_jobs=-1)
grid_search.fit(X, y)

Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed:  6.6min remaining:  4.0min
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:  8.1min finished


CPU times: user 6min 6s, sys: 498 ms, total: 6min 6s
Wall time: 14min 10s


GridSearchCV(cv=2,
             estimator=SVC(gamma='auto', probability=True, random_state=0),
             n_jobs=-1,
             param_grid={'C': [0.001, 10.0], 'kernel': ['rbf', 'poly']},
             verbose=2)

In [11]:
grid_search.best_params_, grid_search.best_score_

({'C': 10.0, 'kernel': 'rbf'}, 0.91288)

In [12]:
#estimator.score(X, y)

In [13]:
#accuracy

In [14]:
# summarize results
#print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
#means = grid_result.cv_results_['mean_test_score']
#stds = grid_result.cv_results_['std_test_score']
#params = grid_result.cv_results_['params']
#for mean, stdev, param in zip(means, stds, params):
#    print("%f (%f) with: %r" % (mean, stdev, param))

In [15]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [16]:
from dask.distributed import Client
tritonCluster = Client("tcp://192.168.86.34:8786")

In [17]:
tritonCluster

Client Scheduler: tcp://192.168.86.34:8786 Dashboard: http://192.168.86.34:8787/status,Cluster Workers: 2 Cores: 14 Memory: 21.82 GB


In [20]:
import joblib
#from sklearn.model_selection import GridSearchCV
#from sklearn.svm import SVC

#estimator = SVC(gamma='auto', random_state=0, probability=True)
#param_grid = {"C": [0.001, 0.1, 1.0, 2.5, 5, 10.0]}

grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=5, n_jobs=-1)

In [ ]:
%%time
with joblib.parallel_backend("dask", scatter=[X, y]):
    grid_search.fit(X, y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend DaskDistributedBackend with 14 concurrent workers.


In [22]:
grid_search.best_params_, grid_search.best_score_

({'C': 2.5}, 0.9139199999999998)